In [1]:
import networkx as nx

In [23]:
def find_density(G, U):
    H = G.subgraph(U)
    return len(H.edges())/len(H.nodes())


In [10]:
find_density(nx.path_graph(10), [i for i in range(5)])

0.8

In [11]:
find_density(nx.path_graph(10), [i for i in range(0, 10, 2)])

0.0

In [12]:
find_density(nx.complete_graph(10), [i for i in range(10)])

4.5

In [24]:
find_density(nx.complete_graph(10), [i for i in range(5)])

2.0

In [25]:
import math

def maximum_density(G):
    #print(G.nodes())
    max_dens = 0
    max_U = []
    for i in range(1, int(math.pow(2, len(G.nodes())))):
        #print("i", i)
        cmb = list(map(int, bin(i)[2:]))
        if len(cmb)<len(G.nodes()):
            cmb = [0 for i in range(len(G.nodes())-len(cmb))] + cmb
        #print(cmb)
        U = []
        i = 0
        for n in G.nodes():
            if cmb[i]==1:
                U.append(n)
            i += 1
        #print(U)
        t = find_density(G, U)
        if t>max_dens:
            max_dens = t
            max_U = U
    return max_dens, max_U

maximum_density(nx.path_graph(5))


(0.8, [0, 1, 2, 3, 4])

In [26]:
def ordered_edge(u, v):
    if u>v:
        t = u
        u = v
        v = t
    return u, v

In [27]:
def restricted_to_edges(E, U, G):
    H = G.subgraph(U)
    E_H = set()
    for e in H.edges():
        u, v = e
        u, v = ordered_edge(u, v)
        E_H.add((u, v))
    E_E = set()
    for e in E:
        u, v = e
        u, v = ordered_edge(u, v)
        E_E.add((u, v))
    return E_H & E_E

restricted_to_edges([(1, 0), (1, 2)], [0, 1, 2], nx.complete_graph(4))

{(0, 1), (1, 2)}

In [28]:
def cov_G(E, v, G):
    N_v = G[v]
    U = []
    for u in N_v:
        U.append(u)
    return len(restricted_to_edges(E, U, G))
    
cov_G([(1, 0), (1, 2)], 2, nx.path_graph(4))

0

In [29]:
def restricted_neighbor_graph(E, v, G):
    # get nodes
    N_v = G[v]
    U = []
    for u in N_v:
        U.append(u)
    # add edges
    G_out = nx.Graph()
    G_out.add_nodes_from(U)
    H = G.subgraph(U)
    for e in H.edges():
        u, v = e
        G_out.add_edge(u, v)
    # remove edges
    E_st = set()
    for e in E:
        u, v = e
        u, v = ordered_edge(u, v)
        E_st.add((u, v))
    G_st = set()
    for e in G_out.edges():
        u, v = e
        u, v = ordered_edge(u, v)
        G_st.add((u, v))
    E2 = G_st - E_st
    for e in E2:
        u, v = e
        G_out.remove_edge(u, v)
    return G_out

restricted_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.path_graph(4))

In [30]:
def max_density_neighbor_graph(E, v, G):
    return maximum_density(restricted_neighbor_graph(E, v, G))

#max_density_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.path_graph(4))
max_density_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.complete_graph(4))

(0.5, [0, 1])

In [40]:
def approx_2_spanner(G):
    ordered_edges = []
    for e in G.edges():
        u, v = e
        u, v = ordered_edge(u, v)
        ordered_edges.append((u, v))
    H_u = set(ordered_edges)
    H_c = set()
    H_s = set()
    #print(H_u, H_c, H_s)
    while True:
        mx_density = 0
        mx_U = -1
        mx_v = -1
        for v in G.nodes():
            density, U_v = max_density_neighbor_graph(H_u, v, G)
            if mx_density < density:
                mx_density = density
                mx_U = U_v
                mx_v = v
        #print(mx_density)
        if mx_density<1:
            break
        #update H_u, H_c, H_s
        E_new = set()
        for u in mx_U:
            x = u
            y = mx_v
            x, y = ordered_edge(x, y)
            E_new.add((x, y))
        H_s = H_s.union(E_new)
        H_c = H_c.union(restricted_to_edges(H_u, mx_U, G)).union(H_s)
        H_u = H_u - H_c
    return H_s.union(H_u)
    
G = nx.complete_graph(4)
print(approx_2_spanner(G))
print(G.edges())
G = nx.wheel_graph(10)
print(approx_2_spanner(G))
print(G.edges())

{(0, 1), (0, 3), (0, 2)}
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
{(0, 1), (0, 7), (0, 6), (0, 5), (0, 4), (0, 9), (0, 3), (0, 8), (0, 2)}
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 9), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]
